In [1]:
!pip install xmltodict

In [13]:
import xmltodict
import json
import os
import shutil

In [14]:
#label_dict
label = ['bicycle', 'bus', 'car', 'carrier', 'cat', 'dog', 'motorcycle', 
        'movable_signage', 'person', 'scooter', 'stroller', 'truck', 'wheelchair', 
        'barricade', 'bench', 'bollard', 'chair', 'fire_hydrant', 'kiosk', 'parking_meter', 
        'pole', 'potted_plant', 'power_controller', 'stop', 'table', 'traffic_light', 
        'traffic_light_controller', 'traffic_sign', 'tree_trunk']
label_dict = { label_name: idx for idx, label_name in enumerate(label)}

In [15]:
src_dir = '/opt/ml'
dst_dir = "/opt/ml/yolov7/yolov7/data/val"
xml_list = []
for i in range(5,6):
    dir1 = os.path.join(src_dir,f"Bbox_{i}_new")
    dir_list = os.listdir(dir1)
    for dir2 in dir_list:
        dir3 = os.path.join(dir1,dir2)
        file_list = os.listdir(dir3)
        for file_name in  file_list:
            if ".xml" in file_name:
                xml_list.append(os.path.join(dir3,file_name))
        

In [22]:
for xml_path in xml_list:
    with open(xml_path, 'rt', encoding="UTF-8") as f:
        doc = xmltodict.parse(f.read())
    json_data = json.loads(json.dumps(doc))
    for image in json_data['annotations']['image']:
        # 파일 만들고
        label_path = os.path.join(dst_dir,"labels")
        file_name = image['@name'].split('.')[0]
        image_w = int(image['@width'])
        image_h = int(image['@height'])
        f = open(os.path.join(label_path,file_name+".txt"), 'w',encoding='UTF-8')

        if isinstance(image['box'], dict):
            xl = float(box['@xtl'])
            xr = float(box['@xbr'])
            yt = float(box['@ytl'])
            yb = float(box['@ybr'])
            x_center = ((xl+xr)/2)
            x_center = x_center/image_w
            y_center = ((yt+yb)/2)
            y_center = y_center/image_h
            w = (xr-xl)/image_w
            h = (yb-yt)/image_h
            x_center = format(x_center, '.6f')
            y_center = format(y_center, '.6f')
            w = format(w, '.6f')
            h = format(h, '.6f')
            label = label_dict[box['@label']]
            arr = list(map(str, [label, x_center, y_center, w, h]))
            data = ' '.join(arr) + '\n'
            f.write(data)      
        else:
            for box in image['box']:
                xl = float(box['@xtl'])
                xr = float(box['@xbr'])
                yt = float(box['@ytl'])
                yb = float(box['@ybr'])
                x_center = ((xl+xr)/2)
                x_center = x_center/image_w
                y_center = ((yt+yb)/2)
                y_center = y_center/image_h
                w = (xr-xl)/image_w
                h = (yb-yt)/image_h
                x_center = format(x_center, '.6f')
                y_center = format(y_center, '.6f')
                w = format(w, '.6f')
                h = format(h, '.6f')
                label = label_dict[box['@label']]
                arr = list(map(str, [label, x_center, y_center, w, h]))
                data = ' '.join(arr) + '\n'
                f.write(data)
        f.close()

In [23]:
image_path = os.path.join(dst_dir,"images")
for xml_path in xml_list:
    path = '/'.join(xml_path.split('/')[:-1])
    img_list = os.listdir(path)
    for file_name in img_list:
        if ".xml" in file_name:
            continue
        src_path = os.path.join(path, file_name)
        dst_path = os.path.join(image_path, file_name)
        shutil.copy(src_path, dst_path)